In [1]:
import os

# đọc video từ thư mục data/video
video_path = "data/video/"
video_list = os.listdir(video_path)

In [2]:
# thêm vào danh sách video các video lật ngược name+f
new_video_list = video_list.copy()

for video in video_list:
    new_video_list.append(video[:-4] + "f.mp4")

video_list = new_video_list

In [3]:
import cv2
import sys

sys.path.append("../module/")
from init import init_csv
from pose_landmarker import extract_pose_features

list = []
for video_name in video_list:
    # Nếu tên video có chữ f ở cuối thì xóa chữ f đi và lật ngược video
    if video_name[-5] == "f":
        # Đọc video
        cap = cv2.VideoCapture(os.path.join(video_path, video_name[:-5] + ".mp4"))
    else:
        # Đọc video
        cap = cv2.VideoCapture(os.path.join(video_path, video_name))

    # Kiểm tra xem video có được mở thành công không
    if not cap.isOpened():
        print("Không thể mở video.")
        exit()

    # Khởi tạo csv_writer
    csv_writer = init_csv(f"data/csv/{video_name.split('.')[0]}.csv")

    # Đọc từng frame từ video
    while True:
        ret, frame = cap.read()

        # Kiểm tra nếu có chữ f ở cuối tên video thì lật ngược frame
        if video_name[-5] == "f":
            frame = cv2.flip(frame, 1)

        # Kiểm tra xem có còn frame nào để đọc không
        if not ret:
            break

        # Trích xuất đặc trưng từ frame
        landmarks = extract_pose_features(frame)

        if landmarks is not None:
            # Chuyển landmarks thành mảng 1 chiều
            list = [j for i in landmarks for j in i]
            # Ghi dữ liệu xuống file csv
            csv_writer.writerow(list)

        # Thời gian chờ giữa các frame (để đảm bảo đọc video với tốc độ 30 fps)
        wait_time = int(1000 / 30)

# Đóng cửa sổ hiển thị và giải phóng tài nguyên
cv2.destroyAllWindows()

In [4]:
import pandas as pd
import csv

# Khởi tạo csv_writer
csv_file = open("data/data.csv", "w", newline="")  # Tạo file csv
csv_writer = csv.writer(csv_file)  # Khởi tạo csv_writer

# Khởi tạo header cho file csv
header = [f"{i}" for i in range(0, 990)] + ["label"]
csv_writer.writerow(header)

for video_name in video_list:
    # Đọc file csv
    csv_reader = pd.read_csv(f"data/csv/{video_name.split('.')[0]}.csv")

    # Lấy 28 dòng đầu tiên của file csv và chuyển nó thành mảng 1 chiều
    list = [j for i in csv_reader.head(30).values for j in i]

    # Nếu tên video có số 0 đầu thì thêm số 0 và cuối list
    if video_name.split(".")[0][0] == "0":
        list.append(0)
    else:
        list.append(1)

    # Ghi dữ liệu xuống file csv
    csv_writer.writerow(list)

# Đóng file csv
csv_file.close()

In [5]:
# Đọc file data.csv
csv_reader = pd.read_csv("data/data.csv")

# In ra 5 dòng đầu tiên của file csv
csv_reader

,0,1,2,3,4,5,6,7,8,9,...,981,982,983,984,985,986,987,988,989,label
0,0.479957,0.228102,-0.181908,0.482556,0.301600,-0.016666,0.521695,0.401496,-0.137361,0.441226,...,0.439851,0.642006,-0.051918,0.517081,0.780251,0.096666,0.431878,0.785392,0.086423,0
1,0.472596,0.220750,-0.236944,0.474785,0.292848,-0.047951,0.565683,0.319175,-0.159357,0.384925,...,0.437203,0.639434,-0.032616,0.497706,0.798847,0.078273,0.433351,0.776532,0.095198,0
2,0.490978,0.199762,-0.170836,0.491824,0.284425,0.014305,0.551431,0.374622,-0.047448,0.423966,...,0.455252,0.654917,-0.032976,0.496907,0.795347,0.145337,0.459017,0.794917,0.133280,0
3,0.475542,0.208120,-0.187014,0.477825,0.290468,-0.051541,0.544644,0.304297,-0.246757,0.404366,...,0.429451,0.639583,-0.033117,0.499803,0.796165,0.098819,0.406445,0.783485,0.065370,0
4,0.480069,0.207963,-0.183100,0.484249,0.292227,-0.006678,0.531287,0.384569,0.000923,0.431638,...,0.455878,0.653487,-0.052929,0.506503,0.789057,0.110457,0.462058,0.793928,0.118302,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.456053,0.394601,0.010737,0.413066,0.471565,0.064251,0.377261,0.380748,0.096486,1
182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.725702,0.541282,-0.301577,0.811620,0.503403,-0.168528,0.812246,0.529177,-0.098784,1
183,0.528608,0.487869,-0.485487,0.540305,0.497453,-0.267193,0.570663,0.443873,-0.220234,0.587342,...,0.653030,0.526789,-0.178518,0.737073,0.484890,-0.052340,0.728140,0.507960,-0.162783,1
184,0.517781,0.538693,-0.523041,0.487860,0.541461,-0.444910,0.500384,0.497491,-0.407398,0.446859,...,0.530016,0.541363,-0.305910,0.608791,0.555258,-0.041225,0.598199,0.565399,-0.222106,1
